In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import os
import re

In [2]:
def get_trm():
    response = requests.get('https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/latest/currencies/usd/cop.json')

    if response.status_code == 200:
        data = response.json()
        return (data['cop']) # Imprime el valor de la tasa de cambio
    else:
        print("Ha ocurrido un error al hacer la solicitud.")
        return 5000
        

In [3]:
##Leer la pagina de busqueda para los 16 forros mas buscados ordenados por pedidos y envio gratis si supera la marca
browser = uc.Chrome()

url = 'https://www.amazon.com/s?k=ssd&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&language=es&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&qid=1681448085&rnid=10236241011&sprefix=ssd%2Caps%2C262&ref=sr_st_exact-aware-popularity-rank&ds=v1%3A3Tr9HRE8Yp%2BD%2BnwryWhRB8sFgsNLs9Gf0oAK6kTOXPk'
url2 = 'https://www.amazon.com/-/es/s?k=ssd&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&page=2&language=es&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&qid=1681448126&rnid=10236241011&sprefix=ssd%2Caps%2C262&ref=sr_pg_2'
browser.get(url)

browser.implicitly_wait(60)

html = browser.page_source

soup = bs(html,'lxml')

browser.get(url2)

browser.implicitly_wait(69)

html2 = browser.page_source

soup2 = bs(html,'lxml')

#Cerrar pestaña y navegador
browser.close()
browser.quit()

In [4]:
#Crear array con las tarjetas de cada uno de los elementos (la tarjeta 0 es informacion)
cards = soup.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards2 = soup2.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards = cards + cards2
cards


[<div class="a-section a-spacing-small a-spacing-top-small"><div class="a-section a-spacing-none puis-padding-right-small s-title-instructions-style"><h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/-/es/PCIe-NVMe-interno-juegos-MZ-V8P2T0B/dp/B08RK2SR23/ref=sr_1_1?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&amp;crid=2HNB4W2XNM2PC&amp;keywords=ssd&amp;qid=1681497984&amp;refinements=p_n_is_free_shipping%3A10236242011&amp;rnid=10236241011&amp;sprefix=ssd%2Caps%2C262&amp;sr=8-1"><span class="a-size-medium a-color-base a-text-normal">SSD M.2 980 PRO 2TB PCIe NVMe Gen4 interno para juegos (MZ-V8P2T0B/AM)</span> </a> </h2></div><div class="a-section a-spacing-none a-spacing-top-micro"><div class="a-row a-size-small"><span aria-label="4.8 de 5 estrellas"><span class="a-declarative" data-a-popover='{"closeButton":false,"closeButtonLabel":"","position":"triggerBottom","popoverLabel"

In [5]:
def extract_discount(text):
    # Definir la expresión regular
    regex = r"Ahorra\s+(\d+)\s*%"

    # Buscar el número usando la expresión regular
    match = re.search(regex, text)

    # Devolver el número como un entero si se encontró una coincidencia
    if match:
        return int(match.group(1))

    # Devolver None si no se encontró una coincidencia
    return None


In [6]:
#Definir funciones para extraer info primero de la card y luego del link de la card

def get_name(card):
    name = card.find('span', class_='a-size-medium a-color-base a-text-normal').text
    return name
def get_price(card):
    price = soup.find('span',{'class':'a-price-whole'}).text + soup.find('span',{'class':'a-price-fraction'}).text
    return float(price)*get_trm()
def get_discount(card):
    discount_text = soup.find('span',{'class':'a-size-base s-highlighted-text-padding aok-inline-block s-coupon-highlight-color'}).text
    discount = extract_discount(discount_text)
    return discount
def get_link(card):
    link = card.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    href = 'https://www.amazon.com.co' + link.get('href')
    return href
   

## Ahora a extraer de cada pagina

In [7]:
ssd_test_link = get_link(cards[0])
ssd_test_link

'https://www.amazon.com.co/-/es/PCIe-NVMe-interno-juegos-MZ-V8P2T0B/dp/B08RK2SR23/ref=sr_1_1?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&keywords=ssd&qid=1681497984&refinements=p_n_is_free_shipping%3A10236242011&rnid=10236241011&sprefix=ssd%2Caps%2C262&sr=8-1'

In [8]:
# Abrir la pagina web
PATH = '../usr/lib/chromium-browser/'
driver = webdriver.Chrome(PATH)
driver.get(ssd_test_link)
#driver.maximize_window()

/tmp/ipykernel_3095/3232571169.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [10]:
size_data = driver.find_element(By.XPATH,'//*[@id="tp-inline-twister-dim-values-container"]/ul')
print(list(size_data))

TypeError: 'WebElement' object is not iterable